In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow
!pip install tensorflow-text
#!pip install spacy
!pip install contractions
!pip install emoji
!pip install emot
!pip install demoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 29.3 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      S

In [ ]:
#import spacy

import pandas as pd
import numpy as np
import string
import contractions
#from nltk.stem import WordNetLemmatizer
#from nltk.stem import PorterStemmer
import re
import regex
import demoji
import emoji
from sklearn.metrics import f1_score
import os
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('whitespace')
#nltk.download('wordnet')
#nltk.download('omw-1.4')
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Error loading whitespace: Package 'whitespace' not found
[nltk_data]     in index


In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/Hope_Task2_Span/train.csv')
df_val = pd.read_csv('/content/drive/MyDrive/Hope_Task2_Span/val.csv')

In [ ]:
df_train

,text,binary,multiclass,id
0,Mientras me persigno y le rezo a la Virgen del...,Not Hope,Not Hope,5533
1,No.. Yo ya no estoy para esperar.. Ni para rog...,Not Hope,Not Hope,549
2,Creo que estamos ante el mejor episodio de est...,Hope,Generalized Hope,4350
3,Bueno ojalá q cuando llegue este fin de semana...,Hope,Realistic Hope,3593
4,#USER# #USER# Marcos de boludo no tiene un pel...,Not Hope,Not Hope,770
...,...,...,...,...
6898,#USER# buenos días bella. De nuevo la saludo c...,Hope,Generalized Hope,7945
6899,🔴 GASTRO | I Jornadas de los Gazpachos en Bar ...,Not Hope,Not Hope,4240
6900,La nuera le contó al Dr Petro todas las irregu...,Not Hope,Not Hope,5783
6901,No sé por qué no me siento listo para una rela...,Not Hope,Not Hope,9045


In [ ]:
df_train.multiclass.value_counts()

Not Hope            4701
Generalized Hope    1151
Unrealistic Hope     546
Realistic Hope       505
Name: multiclass, dtype: int64

In [ ]:
english_stopwords = stopwords.words('english')
exclude = set(string.punctuation)

def emo(text):
  temp=emoji.demojize(text,delimiters=(" "," "))
  temp=temp.replace("_","  ")
  return temp

def preprocess(text):
  #text=demoji.findall(df['Text'])
  text = contractions.fix(text.lower(), slang=True)
  text =re.sub("@ ?[A-Za-z0-9_]+", "", text)
  text= re.sub(r'\d+', '', text)
  text=re.sub(r'$', '', text)
  text= re.sub(r'’','', text )
  text=re.sub('<.*?>','',text)
  text=re.sub(r'http\S+', '', text)
  #text=emoji.demojize(text, delimiters=(" ", " "))
  text = ''.join(ch for ch in text if ch not in exclude)
  tokens = word_tokenize(text)
  #print("Tokens:", tokens)
  text = [t for t in tokens if t not in english_stopwords]
  text = " ".join(text)
  return text

def text_preprocess(text):
  temp = []
  for line in sent_tokenize(text):
    temp.append(preprocess(line))
  return temp

In [ ]:
df_train['clean_text']=df_train['text'].apply(lambda x:emo(x))
df_train['clean_text']=df_train['clean_text'].apply(lambda X: text_preprocess(X))

df_val['clean_text']=df_val['text'].apply(lambda x:emo(x))
df_val["clean_text"]=df_val['clean_text'].apply(lambda X: text_preprocess(X))

In [ ]:
from sklearn.utils import resample
#create two different dataframe of majority and minority class
df_majority = df_train[(df_train['multiclass']=='Not Hope')]
df_minority1 = df_train[(df_train['multiclass']=='Generalized Hope')]
df_minority2 = df_train[(df_train['multiclass']=='Unrealistic Hope')]
df_minority3 = df_train[(df_train['multiclass']=='Realistic Hope')]

# upsample minority class
df_minority_upsampled1 = resample(df_minority1,replace=True, n_samples= 4701, random_state=None)  # reproducible results
df_minority_upsampled2 = resample(df_minority2,replace=True, n_samples= 4701, random_state=None)
df_minority_upsampled3 = resample(df_minority3,replace=True, n_samples= 4701, random_state=None)

# Combine majority class with upsampled minority class
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_minority_upsampled1,df_minority_upsampled2,df_minority_upsampled3, df_majority])

In [ ]:
df_upsampled

,text,binary,multiclass,id,clean_text,indic_sim_vec,xlm_vec
31,"Quizá ella podrá ser más bonita que yo, pero y...",Hope,Generalized Hope,5337,[quizá ella podrá ser más bonita que yo pero y...,"[0.01383284479379654, -0.01179566141217947, -0...","[-0.393829882144928, -0.556420087814331, -0.02..."
3613,#USER# Me queda claro y de acuerdo. El punto e...,Hope,Generalized Hope,4580,"[user queda claro de acuerdo, el punto es… red...","[0.010937111685052514, -0.00973542535211891, -...","[-0.11312003433704376, -0.3813704699277878, 0...."
4695,#USER# #USER# Un gol (799) y asistencia (299) ...,Hope,Generalized Hope,434,[user user un gol asistencia contra el bayern ...,"[0.02696031518280506, -0.013328642584383488, -...","[0.5113272666931152, -0.1026657447218895, -0.4..."
6299,#USER# #USER# pobre coty la envidia que le tie...,Hope,Generalized Hope,5464,[user user pobre coty la envidia que le tiene ...,"[0.001474036995205097, -0.009494821541011333, ...","[-0.04893113672733307, -0.7998988628387451, 0...."
5855,#USER# #USER# #USER# #USER# muchas gracias por...,Hope,Generalized Hope,8391,[user user user user muchas gracias por sus or...,"[0.002359022075931231, -0.01841760240495205, -...","[0.040891860922177635, -0.15218881393472353, -..."
...,...,...,...,...,...,...,...
6896,#USER# no dormi pero es por q no tengo sueño f...,Not Hope,Not Hope,2324,[user dormi pero es por q tengo sueño fkkdiwjd],"[0.014890301041305065, -0.0023424287792295218,...","[-1.521673560142517, -0.9886399507522583, 0.01..."
6899,🔴 GASTRO | I Jornadas de los Gazpachos en Bar ...,Not Hope,Not Hope,4240,[red circle gastro jornadas de los gazpachos e...,"[0.010274059139192104, 0.0005360814393498003, ...","[0.6340965628623962, -0.022585248574614525, 0...."
6900,La nuera le contó al Dr Petro todas las irregu...,Not Hope,Not Hope,5783,[la nuera le contó al dr petro todas las irreg...,"[0.0010550105556224783, -0.011642228724667802,...","[0.15053856869538626, -0.1423801127821207, -0...."
6901,No sé por qué no me siento listo para una rela...,Not Hope,Not Hope,9045,[sé por qué siento listo para una relación al ...,"[0.016887377947568893, -0.002686064690351486, ...","[-0.8638034462928772, -0.3621791899204254, -0...."


In [ ]:
# Import label encoder
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

# Encode labels in column 'category'
df_upsampled['sp_label']= le.fit_transform(df_upsampled['multiclass'])
df_val['sp_label']= le.fit_transform(df_val['multiclass'])

#train['en_label'].unique()
#train['en_label'].value_counts()

#le.inverse_transform([2, 2, 1]

In [ ]:
print(le.inverse_transform([0, 1, 2,3]))

['Generalized Hope' 'Not Hope' 'Realistic Hope' 'Unrealistic Hope']


In [ ]:
# unique labels in the train set - only 2 labels
Label = pd.Series([0, 1, 2,3])
y_train = np.array(Label.values.tolist())
y_val = df_val['multiclass']


In [ ]:
y_train

array([0, 1, 2, 3])

In [ ]:
len(y_train)

4

In [ ]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, util
import torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.1/163.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
def SentTransf(model):   # returns the specifiec Sentence Transformers
  return SentenceTransformer(model)

In [ ]:
# bert_model = 'l3cube-pune/gujarati-sentence-bert-nli'
#l3cube-pune/tamil-sentence-bert-nli
bert_model = 'mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es'
M1 = SentTransf(bert_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/135 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
# bert_model = 'l3cube-pune/gujarati-sentence-bert-nli'
bert_model = 'hiiamsid/sentence_similarity_spanish_es'
M1 = SentTransf(bert_model)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.51k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# bert_model = 'l3cube-pune/gujarati-sentence-bert-nli' l3cube-pune/indic-sentence-similarity-sbert
bert_model = 'bertin-project/bertin-roberta-base-spanish'
M2 = SentTransf(bert_model)

config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at bertin-project/bertin-roberta-base-spanish and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/851k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/509k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.21M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [ ]:
def get_embeddings_sent(text, model):
  #print(text)
  embds = model.encode(text)
  return embds

def get_embeddings(text, model):
  #print('text : ', text, 'len :', len(text))
  temp =  np.zeros(768)
  for line in text:   #sent_tokenize(text):
    #print('line : ', line)
    if len(line)>0:
      temp = np.add(temp, get_embeddings_sent(line, model))
  temp = np.divide(temp, len(text))
  #print(temp)
  return temp

In [ ]:
#get_embeddings(tr_text['clean_text'][0])
#df_upsampled['bert_vec'] = df_upsampled['clean_text'].apply(lambda x: get_embeddings(x, M1))
df_upsampled['indic_sim_vec'] = df_upsampled['clean_text'].apply(lambda x: get_embeddings(x, M2))
df_upsampled['xlm_vec'] = df_upsampled['clean_text'].apply(lambda x: get_embeddings(x, M1))

In [ ]:
#df_val['bert_vec'] = df_val['clean_text'].apply(lambda x: get_embeddings(x, M1))
df_val['indic_sim_vec'] = df_val['clean_text'].apply(lambda x: get_embeddings(x, M2))
df_val['xlm_vec'] = df_val['clean_text'].apply(lambda x: get_embeddings(x, M1))

In [ ]:
df_train['multiclass'].unique()

array(['Not Hope', 'Generalized Hope', 'Realistic Hope',
       'Unrealistic Hope'], dtype=object)

In [ ]:
df_train.head()

,text,binary,multiclass,id,clean_text,sp_label
0,Mientras me persigno y le rezo a la Virgen del...,Not Hope,Not Hope,5533,[mientras persigno le rezo la virgen del carme...,1
1,No.. Yo ya no estoy para esperar.. Ni para rog...,Not Hope,Not Hope,549,[yo ya estoy para esperar ni para rogar ni com...,1
2,Creo que estamos ante el mejor episodio de est...,Hope,Generalized Hope,4350,[creo que estamos ante el mejor episodio de es...,0
3,Bueno ojalá q cuando llegue este fin de semana...,Hope,Realistic Hope,3593,[bueno ojalá q cuando llegue este fin de seman...,2
4,#USER# #USER# Marcos de boludo no tiene un pel...,Not Hope,Not Hope,770,[user user marcos de boludo tiene un pelo pero...,1


In [ ]:
Xtrain1 = df_upsampled['xlm_vec']  #.to_numpy()  #.apply(pd.Series).reset_index()m4
Xtrain1 = np.array(Xtrain1.values.tolist())
Xtrain2 = df_upsampled['indic_sim_vec']  #.to_numpy()  #.apply(pd.Series).reset_index()m4
Xtrain2 = np.array(Xtrain2.values.tolist())

In [ ]:
# Xtrain3 = df_train['distil_vec']  #.to_numpy()  #.apply(pd.Series).reset_index()m4
# Xtrain3 = np.array(Xtrain3.values.tolist())

In [ ]:
y_train = df_upsampled['sp_label']
y_train = np.array(y_train.values.tolist())


In [ ]:
Xtest1 = df_val['xlm_vec']  #.to_numpy()  #.apply(pd.Series).reset_index()m4
Xtest1 = np.array(Xtest1.values.tolist())
Xtest2 = df_val['indic_sim_vec']  #.to_numpy()  #.apply(pd.Series).reset_index()m4
Xtest2 = np.array(Xtest2.values.tolist())

In [ ]:
# Xtest3 = df_val['distil_vec']  #.to_numpy()  #.apply(pd.Series).reset_index()m4
# Xtest3 = np.array(Xtest3.values.tolist())

In [ ]:
y_val = df_val['sp_label']
y_val = np.array(y_val.values.tolist())

In [ ]:
from sklearn.utils import all_estimators

#estimators = all_estimators()

#for name, class_ in estimators:
#    if hasattr(class_, 'predict_proba'):
#        print(name)

estimators = all_estimators(type_filter='classifier')
for name, class_ in estimators:
    module_name = str(class_).split("'")[1].split(".")[1]
    class_name = class_.__name__
    print(f'from sklearn.{module_name} import {class_name}')

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import CategoricalNB
from sklearn.multioutput import ClassifierChain
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
fr

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

clf5 = LogisticRegression(random_state=1)   # have to increase the number of repeatitions
clf1 = SVC(probability=True)  #DecisionTreeClassifier()   #GaussianNB()  # BernoulliNB()
clf4 = LinearSVC(penalty='l2', C=1.0)
clf2 = RandomForestClassifier(max_depth=2, random_state=0)

In [ ]:
len(Xtest1), len(Xtest2), len(Xtrain1), len(Xtrain2)

(1150, 1150, 18804, 18804)

In [ ]:
clf1 = clf1.fit(Xtrain1, y_train)
y_pred1 = clf1.predict(Xtest1)
y_pp1 = clf1.predict_proba(Xtest1)

In [ ]:
y_pp1

array([[2.04591442e-03, 9.53923518e-01, 5.51125815e-04, 4.34794421e-02],
       [2.74300820e-02, 9.72546887e-01, 2.14404959e-05, 1.59035796e-06],
       [3.59498648e-01, 6.37342376e-01, 2.53874571e-03, 6.20230545e-04],
       ...,
       [8.50631137e-04, 9.91293451e-01, 1.92355865e-04, 7.66356176e-03],
       [1.64339812e-01, 8.27369065e-01, 1.34821641e-03, 6.94290696e-03],
       [8.59438949e-04, 9.97423437e-01, 9.42302746e-05, 1.62289337e-03]])

In [ ]:
y_pp1_max_ind = y_pp1.argmax(axis=1)   # to get the index of the maximum value
y_pp1_max_p = np.amax(y_pp1, axis=1)   # to get the maximum probability value

In [ ]:
clf2 = clf2.fit(Xtrain2, y_train)
y_pred2 = clf2.predict(Xtest2)
y_pp2 = clf2.predict_proba(Xtest2)

In [ ]:
y_pp2

array([[0.21558519, 0.25496382, 0.25157992, 0.27787107],
       [0.27874018, 0.25174398, 0.21884336, 0.25067248],
       [0.27989158, 0.24390818, 0.23661253, 0.23958772],
       ...,
       [0.22397812, 0.26333575, 0.26262975, 0.25005638],
       [0.26580357, 0.25315782, 0.25559891, 0.22543969],
       [0.22160348, 0.26244494, 0.27261396, 0.24333761]])

In [ ]:
y_pp2_max_ind = y_pp2.argmax(axis=1)   # to get the index of the maximum value
y_pp2_max_p = np.amax(y_pp2, axis=1)   # to get the maximum probability value

In [ ]:
def get_pred(y_pred1, y_pp1_max_p, y_pred2, y_pp2_max_p):
  pred = []
  for i in range(len(y_pred1)):
    if y_pred1[i] == y_pred2[i]:
      pred.append(y_pred1[i])
    else:
      if y_pp1_max_p[i] >= y_pp2_max_p[i]:
        pred.append(y_pred1[i])
      else:
        pred.append(y_pred2[i])
  return pred

In [ ]:
pred = get_pred(y_pred1, y_pp1_max_p, y_pred2, y_pp2_max_p)
pred_labels = list(le.inverse_transform(pred))
print(classification_report(df_val['multiclass'], pred_labels))  #, target_names=target_names)

                  precision    recall  f1-score   support

Generalized Hope       0.53      0.59      0.56       186
        Not Hope       0.84      0.84      0.84       799
  Realistic Hope       0.26      0.23      0.24        74
Unrealistic Hope       0.38      0.31      0.34        91

        accuracy                           0.72      1150
       macro avg       0.50      0.49      0.50      1150
    weighted avg       0.72      0.72      0.72      1150



In [ ]:
y_pred11 = clf1.predict(Xtest1)
y_pred22 = clf2.predict(Xtest2)

In [ ]:
y_pp11 = clf1.predict_proba(Xtest1)
y_pp1_max_p1 = np.amax(y_pp11, axis=1)
y_pp22 = clf2.predict_proba(Xtest2)
y_pp1_max_p2 = np.amax(y_pp22, axis=1)

In [ ]:
pred = get_pred(y_pred11, y_pp1_max_p1, y_pred22, y_pp1_max_p2)
pred_labels = list(le.inverse_transform(pred))

In [ ]:
pred_labels

['Not Hope',
 'Not Hope',
 'Not Hope',
 'Not Hope',
 'Not Hope',
 'Not Hope',
 'Not Hope',
 'Not Hope',
 'Not Hope',
 'Not Hope',
 'Not Hope',
 'Not Hope',
 'Unrealistic Hope',
 'Not Hope',
 'Not Hope',
 'Not Hope',
 'Not Hope',
 'Not Hope',
 'Not Hope',
 'Not Hope',
 'Not Hope',
 'Not Hope',
 'Not Hope',
 'Not Hope',
 'Not Hope',
 'Unrealistic Hope',
 'Not Hope',
 'Not Hope',
 'Generalized Hope',
 'Generalized Hope',
 'Not Hope',
 'Not Hope',
 'Not Hope',
 'Realistic Hope',
 'Not Hope',
 'Generalized Hope',
 'Not Hope',
 'Generalized Hope',
 'Not Hope',
 'Not Hope',
 'Generalized Hope',
 'Not Hope',
 'Not Hope',
 'Not Hope',
 'Not Hope',
 'Not Hope',
 'Generalized Hope',
 'Not Hope',
 'Not Hope',
 'Realistic Hope',
 'Generalized Hope',
 'Not Hope',
 'Not Hope',
 'Not Hope',
 'Generalized Hope',
 'Generalized Hope',
 'Not Hope',
 'Generalized Hope',
 'Not Hope',
 'Not Hope',
 'Not Hope',
 'Generalized Hope',
 'Realistic Hope',
 'Generalized Hope',
 'Not Hope',
 'Not Hope',
 'Generalize

In [ ]:
len(pred_labels),Xtest1.shape

(1150, (1150, 768))

In [ ]:
#Submission
# to create .csv file consisting of tweet ids and predicted labels
y_pred_df = pd.DataFrame(data=pred_labels, columns=['final_prediction'])
Submisssion_sp = pd.DataFrame()
Submisssion_sp['id'] = df_val['id']
Submisssion_sp['category'] = y_pred_df
Submisssion_sp.to_csv('/content/drive/MyDrive/Colab Notebooks/spanmulti_task2.csv',index = None)